In [1]:
import cv2
import numpy as np
import skimage.io as io
from skimage.feature import canny
import matplotlib.pyplot as plt
from skimage.color import rgb2gray , rgb2hsv
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.draw import rectangle
from skimage.filters import threshold_otsu, threshold_mean, threshold_li, threshold_isodata, threshold_niblack , median
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin , disk
from skimage.segmentation import flood, flood_fill
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
def Gamma_Correction(img , c , gamma):
    img2 = np.copy(img)
    img2 = c*np.power(img , gamma)
    return img2

def erosion(img , n,m):
    img2 = np.copy(img)
    for i in range(int(n/2) , img.shape[1]-1-int(n/2) ):
        for j in range(int(m/2) , img.shape[0]-1-int(m/2) ):
            img2[j,i] = np.sum(img[j-int((m)/2) : j+int((m)/2) + n%2 , i-int((n)/2) : i+int((n)/2) + n%2 ])==n*n        
    return img2

def dilation(img , n,m):
    img2 = np.copy(img)
    for i in range(int(n/2) , img.shape[1]-1-int(n/2) ):
        for j in range(int(m/2) , img.shape[0]-1-int(m/2) ):
            img2[j,i] = np.sum(img[j-int((m)/2) : j+int((m)/2) + n%2 , i-int((n)/2) : i+int((n)/2) + n%2 ])>=1        
    return img2

def Median(img , n , m):
    img2 = np.copy(img)
    filt = np.ones((n,m))
    for i in range(int(n/2) , img.shape[1]-int(np.ceil(n/2))):
        for j in range(int(m/2) , img.shape[0]-int(np.ceil(m/2))):
            img2[j,i] = np.median(img[j-int(m/2) : j+int(m/2)+1 , i-int(n/2) : i+int(n/2)+1 ])
    return img2


In [3]:
def multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame):
    
    # Extracting the blue, red, green and alpha channels
    
    B = rgba_frame[:,:,0]
    G = rgba_frame[:,:,1]
    R = rgba_frame[:,:,2]
    A = rgba_frame[:,:,3]
    
    #Extracting the Hue, Saturation and vue channels
    
    H = hsv_frame[:,:,0]
    S = hsv_frame[:,:,1]
    V = hsv_frame[:,:,2]
    
    # Extracting the Y, Cr and Cb channels
    
    Y = ycbcr_frame[:,:,0]
    Cr = ycbcr_frame[:,:,1]
    Cb = ycbcr_frame[:,:,2]


#     Applying  Thresholding using Log-Chromaticity color space 
    n_r = np.ma.divide(R, G)
    n_b = np.ma.divide(B, G)
    log_rg = np.ma.log( n_r )
    log_bg = np.ma.log( n_b )
    condition_1 = (log_rg>=0.15) & (log_rg<=1.1)
    condition_2 = (log_bg>=-4) & (log_bg<=0.3)
    mask_1 = condition_1 & condition_2
    
    # Extracting masks based on a combination of RGBA, HSV and YCrCb models for skin detection
    
    mask_rgb = (R>95)&(G>40)&(B>20)&(R>G)&(R>B)&(abs(R-G)>15)&(A>15)
    mask_Ycbcr = (Cr > 135)&(Cb>85)&(Y>80)&(Cr <= (1.5862*Cb)+20)&(Cr>=(0.3448*Cb)+76.2069)&(Cr >= (-4.5652*Cb)+234.5652)&(
                  Cr <= (-1.15*Cb)+301.75)&(Cr <= (-2.2857*Cb)+432.85)
    mask_hsv = ((0.0 <= H) & (H <= 50.0))&((0.23 <= S) &(S <= 0.68))
    
    mask1 = mask_rgb & mask_Ycbcr
    mask2 = mask_rgb & mask_hsv
    
    return mask1 | mask2
    

In [4]:
def moving_area():
    
    frame_number = 0
    ret, rgb_frame = cap.read()
    rgb_frame = cv2.resize(rgb_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    gray_frame = rgb2gray(rgb_frame) 
    acc_frame = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    
    while (frame_number < 3):
        ret, cur_frame = cap.read()
        cur_frame = cv2.resize(cur_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
        cur_frame = rgb2gray(cur_frame)
        acc_frame += cur_frame
        frame_number += 1 
        
    acc_frame /= frame_number
    diff_frame = abs(acc_frame-gray_frame)
    moving_region = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    moving_region[diff_frame > 0.2] = 1
    one_positions = np.argwhere(moving_region == 1)
    if(len(one_positions) > 0):
        Xmin = min(one_positions[:,0])-20
        Xmax = max(one_positions[:,0])+20
        Ymin = min(one_positions[:,1])-120
        Ymax = max(one_positions[:,1])+120
        moving_region[Xmin:Xmax , Ymin:Ymax] = 1
    
    return moving_region
    

In [5]:
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

old_moving_region = [] 

while True:
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    ret, rgb_frame = cap.read()
    rgb_frame = cv2.resize(rgb_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    
#     cannyimg = canny(rgb2gray(rgb_frame)*255, low_threshold= 50, high_threshold = 90)
#     cannyImg = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
#     cannyImg[cannyimg] = 1
    
    gray = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
#     for (x,y,w,h) in faces: 
#         # To draw a rectangle in a face  
#         cv2.rectangle(rgb_frame,(x,y),(x+w,y+h),(255,255,0),2)   
     
    
    # converting the rgb space to hsv space color
    hsv_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2HSV)
    # converting the rgb space to YCbCr space color
    ycbcr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2YCR_CB)
    
    # converintg the rgb space to rgba spave 
    rgba_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2BGRA)
    
    
    # the final mask to extract the binary image
    mask = multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame)
    
    binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    binary_frame2[mask] = 1
    
#     for (x,y,w,h) in faces: 
#         # To draw a rectangle in a face  
#         binary_frame2[y-30:y+h+30 , x:x+w] = 0
        
    #removing noise
     
    binary_frame2 = binary_dilation(binary_frame2 , np.ones((20,2)))
    binary_frame2 = median(binary_frame2 , disk(3) )
    
    contours, hierarchy = cv2.findContours(binary_frame2, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    bounding_boxes = list()
    mask_face = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        print(x,y,w,h)
        if( (w/h <= 0.9 and h/w >= 1.2) or (h/w <= 0.5 and w/h >= 1.3 ) ):
            mask_face[y:y+h , x:x+w] = 1
            cv2.drawContours(rgb_frame, contour, -1, (0, 255, 0), 3)
    
    #cv2.imshow('binary canny detection', final_frame)
#     cv2.imshow('canny detection', cannyImg )

    frame_with_hand_only = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    frame_with_hand_only[(binary_frame2>=1) & (mask_face>=1) ] = 1
    
    cv2.imshow('Input', rgb_frame)
    cv2.imshow('Output1', binary_frame2)
    cv2.imshow('mask_face', mask_face)
    cv2.imshow('Final Output', frame_with_hand_only)
    
    
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()


517 459 3 16
586 217 10 24
473 210 13 24
605 179 8 26
431 120 187 360
547 408 7 20
537 375 8 12
571 349 5 5
555 349 5 10
556 321 12 24
541 316 8 8
615 74 7 25
42 455 4 19
583 220 5 21
421 121 194 359
499 449 7 6
477 428 5 5
536 316 8 5
449 197 82 46
467 208 17 25
611 73 11 31
33 412 5 21
573 220 6 21
409 121 201 359
448 197 63 46
611 73 11 37
42 460 4 20
49 440 3 16
38 433 4 19
29 396 5 21
561 219 11 24
397 122 209 358
529 336 5 4
444 198 59 45
611 72 13 38
42 455 4 19
71 441 3 16
217 440 3 16
21 379 17 55
137 346 3 17
557 220 6 22
389 121 212 359
440 217 7 9
444 197 51 46
611 72 12 38
557 0 3 8
143 396 5 20
161 389 4 19
15 375 31 103
147 356 5 20
277 346 3 17
155 345 3 17
297 341 5 20
546 219 11 23
405 190 4 18
217 158 5 20
385 120 211 360
513 326 6 9
437 216 5 6
439 196 49 47
609 73 14 36
257 0 3 8
245 471 3 9
75 471 3 9
233 469 5 11
111 463 3 17
57 451 5 29
75 440 3 16
253 429 5 20
207 418 5 21
189 415 13 24
32 406 14 71
235 400 7 27
181 386 5 20
195 384 9 23
20 377 12 27
135 376 5 

183 471 3 9
127 471 3 9
241 470 5 10
97 470 7 10
195 462 3 18
167 461 3 16
219 458 11 22
59 438 3 17
169 436 4 18
38 430 8 50
247 415 5 20
159 415 5 36
171 406 3 17
195 404 11 35
81 403 15 53
115 391 5 22
97 386 11 28
235 383 9 48
223 380 11 67
0 377 6 20
15 375 19 44
221 373 3 16
191 371 6 21
153 369 17 37
211 367 7 21
265 360 7 36
297 349 5 21
283 348 7 19
247 348 15 42
197 346 3 19
189 346 3 17
163 346 19 56
109 346 7 40
85 346 5 21
117 343 27 50
139 341 21 80
319 336 7 22
221 329 15 44
343 328 3 17
103 306 5 21
85 304 3 17
183 249 13 39
421 237 205 243
541 330 14 23
529 329 11 10
558 327 8 18
401 198 4 20
409 166 5 22
221 160 3 17
375 150 7 29
357 141 3 17
367 132 5 21
460 130 152 97
375 119 3 16
611 73 12 31
187 53 5 20
411 47 7 22
97 40 5 20
539 35 3 16
385 28 3 17
545 10 5 20
589 0 3 8
559 0 7 23
541 0 3 8
459 0 3 8
451 0 3 8
369 0 3 8
339 0 13 11
327 0 7 9
249 0 3 8
239 0 3 8
227 0 8 11
209 0 3 8
191 0 3 8
163 0 9 28
119 0 3 8
89 0 3 8
201 471 3 9
181 471 3 9
101 471 3 9
75 471

447 471 3 9
133 471 3 9
121 471 3 9
109 471 3 9
61 471 3 9
55 471 3 9
241 461 5 19
41 449 5 30
75 437 9 22
73 416 3 17
219 413 7 26
179 408 3 16
87 401 3 18
0 395 2 18
155 381 15 51
229 379 7 26
23 378 19 67
99 376 13 41
11 374 7 21
213 372 5 36
207 366 3 16
259 360 3 16
189 345 15 82
229 344 5 21
239 343 21 97
217 343 9 22
115 342 19 69
0 342 2 18
269 341 5 21
203 341 3 17
135 341 15 39
97 341 13 26
165 339 25 27
89 336 5 20
343 328 3 16
447 307 4 19
95 279 3 16
133 270 3 17
81 269 5 24
0 258 2 18
295 251 3 16
375 250 7 21
431 244 193 236
573 324 9 12
461 310 5 10
477 303 88 73
506 289 5 6
219 152 3 17
413 147 5 22
467 138 131 90
269 84 3 16
609 72 13 33
283 70 3 17
527 68 3 16
85 67 5 20
297 65 5 20
395 63 5 21
243 61 5 20
133 60 3 16
359 56 5 20
569 52 3 16
591 51 3 16
231 48 5 20
207 46 3 17
147 45 5 20
409 28 3 17
0 24 2 22
315 20 3 17
587 0 3 8
581 0 3 8
571 0 3 8
517 0 3 8
485 0 5 19
479 0 3 8
447 0 3 8
437 0 3 8
407 0 3 8
399 0 3 8
389 0 3 8
295 0 7 11
279 0 3 8
269 0 3 8
257 0

53 471 3 9
77 438 9 22
113 412 5 21
73 403 3 16
133 395 5 21
85 392 5 23
159 380 13 44
15 376 31 104
433 374 3 21
134 371 3 17
0 368 2 18
171 345 7 22
95 343 27 47
141 340 21 59
148 359 6 15
638 314 2 18
637 270 3 20
171 250 3 17
0 202 2 37
175 194 159 259
217 158 5 20
371 148 3 16
225 140 3 17
397 130 4 20
607 126 25 30
416 98 154 215
443 198 3 3
430 169 67 40
219 90 11 47
143 76 3 17
611 73 12 37
393 64 3 16
559 61 5 20
455 60 3 17
197 58 3 17
105 57 3 17
583 56 5 21
253 48 5 20
403 45 11 34
202 39 4 18
0 35 2 18
311 31 3 16
331 17 5 20
171 17 7 21
581 0 3 8
561 0 5 10
545 0 3 8
455 0 3 8
447 0 3 8
397 0 3 8
389 0 5 10
357 0 3 8
335 0 3 8
295 0 3 8
279 0 3 8
247 0 3 8
233 0 3 8
193 0 3 8
153 0 9 23
125 0 3 8
88 468 6 12
129 459 3 16
121 459 3 16
39 451 7 29
445 435 3 16
149 427 3 17
57 419 3 16
113 396 5 32
19 377 22 64
97 370 9 24
69 369 3 16
0 349 2 18
127 347 3 16
141 343 41 97
107 343 17 48
0 326 2 18
139 323 3 16
525 297 4 19
499 274 23 44
0 238 2 18
0 198 2 18
397 193 3 18
436 

53 471 3 9
69 469 5 11
42 456 4 21
461 455 3 17
73 437 5 22
57 434 5 25
37 430 5 21
139 428 3 16
113 415 3 16
177 412 5 21
567 405 4 21
99 401 3 16
0 380 2 19
17 377 15 30
221 370 3 16
161 369 9 28
173 364 7 33
205 346 3 17
157 346 3 16
183 345 20 67
105 345 11 44
123 341 35 86
217 331 5 33
0 319 2 26
0 279 2 26
189 270 5 37
178 249 4 19
413 175 5 21
221 167 5 20
629 134 5 20
375 133 7 23
241 127 169 275
365 279 5 8
445 102 158 220
0 101 2 19
453 85 4 18
233 76 5 20
611 73 12 36
189 50 5 20
589 49 3 16
263 36 3 17
257 30 3 17
353 29 5 20
237 26 5 20
0 7 2 18
575 0 3 8
567 0 3 8
435 0 5 15
333 0 5 10
305 0 9 10
283 0 3 8
257 0 3 8
219 0 3 8
167 0 5 18
137 0 3 8
37 0 7 9
87 471 3 9
51 471 3 9
562 468 9 12
79 462 5 18
42 460 6 20
461 444 4 18
83 443 3 16
71 440 3 16
38 432 4 21
149 420 5 21
142 419 3 16
130 419 4 19
159 402 3 18
109 400 5 20
127 388 3 26
141 386 5 21
171 383 9 34
17 376 15 30
206 370 3 17
147 352 3 16
133 345 11 32
149 344 23 54
113 344 7 21
177 343 21 43
197 342 11 23
97

...

147 471 3 9
69 471 3 9
53 471 3 9
121 467 3 13
85 452 5 28
40 450 6 30
75 419 3 17
141 418 3 17
197 416 5 21
83 410 9 21
113 407 5 20
171 402 21 54
217 386 11 22
119 381 11 25
153 377 7 33
105 377 7 21
99 377 3 16
7 377 3 16
13 375 25 58
145 373 3 16
202 358 12 36
177 356 9 32
161 347 9 22
151 347 3 16
0 346 2 18
197 345 5 20
115 345 11 30
249 344 21 30
217 344 11 21
279 341 5 20
305 325 4 18
139 312 5 21
191 291 3 16
89 267 5 21
0 231 2 18
553 230 3 16
0 179 2 18
215 142 13 45
637 141 3 20
319 136 309 344
441 298 55 77
607 126 27 30
0 102 2 18
611 73 12 37
0 70 2 18
395 61 3 16
279 58 3 17
575 50 5 21
165 47 5 21
191 38 5 21
235 34 3 17
499 0 3 8
407 0 3 8
347 0 3 8
277 0 3 8
235 0 3 8
201 0 3 8
185 0 7 9
179 0 3 8
147 0 3 8
121 0 3 8
113 0 3 8
83 0 3 8
177 471 3 9
113 471 3 9
83 471 3 9
53 471 3 9
69 469 3 11
217 461 5 19
312 450 13 30
77 438 5 21
85 423 3 16
217 416 3 16
193 413 7 24
23 380 25 100
163 379 27 60
13 375 5 20
203 370 7 40
135 364 7 42
119 358 3 16
257 357 3 16
225 357 

165 471 3 9
125 471 3 9
113 471 3 9
77 469 5 11
79 438 7 23
63 435 5 21
171 410 3 17
177 389 11 48
105 385 3 18
237 384 3 17
25 384 25 96
149 383 13 44
17 377 5 20
0 367 6 29
291 364 343 116
151 363 3 17
251 351 3 16
233 346 11 29
171 344 15 41
157 344 5 21
91 344 7 22
137 343 9 20
119 343 9 28
191 341 39 103
227 330 3 16
195 323 3 16
325 314 5 20
191 282 3 17
91 267 3 16
193 256 3 17
261 251 5 20
299 250 5 20
0 246 2 18
340 212 154 146
0 195 2 18
0 141 2 19
217 140 11 38
343 134 139 88
333 129 5 20
607 126 27 30
341 115 5 20
0 113 2 18
145 78 3 16
161 77 5 20
611 72 13 48
213 63 5 20
225 57 5 20
0 55 2 18
273 36 3 16
0 30 2 18
229 24 5 21
485 16 5 20
431 5 3 16
585 0 3 8
527 0 3 8
495 0 3 8
445 0 5 10
419 0 3 8
385 0 3 8
363 0 3 8
355 0 3 8
343 0 7 9
319 0 3 16
259 0 5 10
245 0 7 10
227 0 3 8
201 0 5 17
171 0 3 8
157 0 3 8
127 0 3 8
95 0 4 9
0 0 2 9
221 471 3 9
193 471 3 9
53 471 3 9
83 467 5 13
183 438 3 17
91 415 5 20
205 408 11 70
83 392 5 20
195 388 3 16
99 384 18 61
149 382 11 35

215 471 3 9
177 471 3 9
117 471 3 9
61 471 3 9
149 464 5 16
85 461 7 19
73 438 7 21
161 434 5 21
395 433 3 16
151 430 3 16
223 420 7 34
169 419 5 21
199 414 3 17
207 404 7 36
107 400 5 20
117 392 3 17
27 388 19 92
165 382 31 51
215 381 7 22
85 379 5 20
11 377 5 17
81 356 5 20
0 354 2 18
259 353 5 20
235 348 9 38
191 346 3 17
245 345 7 20
197 345 13 62
269 344 5 21
157 343 13 41
121 341 9 46
213 340 13 25
97 340 17 27
135 333 7 32
195 317 5 20
394 269 8 28
289 252 3 17
227 251 5 20
409 236 74 133
433 322 8 21
456 286 6 8
27 229 5 20
264 227 132 253
473 222 167 258
217 158 7 21
363 143 142 87
595 132 4 20
609 126 23 30
341 115 9 26
389 90 5 21
445 85 3 16
425 79 5 20
609 73 14 37
359 56 5 20
139 47 5 20
305 35 5 20
407 32 3 17
0 31 2 19
593 0 3 8
585 0 3 8
561 0 3 8
533 0 3 8
515 0 3 8
437 0 3 8
429 0 3 8
363 0 11 10
341 0 3 8
331 0 3 8
311 0 3 8
291 0 3 8
267 0 7 9
225 0 5 23
187 0 3 8
179 0 3 8
165 0 7 9
391 471 3 9
219 471 3 9
173 471 3 9
161 471 3 9
127 471 3 9
107 471 3 9
63 471 3 9

201 471 3 9
73 471 3 9
147 469 5 11
183 459 3 17
69 438 3 16
227 431 4 6
501 423 3 22
217 416 3 16
57 415 7 20
115 407 5 21
225 403 7 21
177 397 19 37
0 390 2 18
113 385 5 21
137 384 3 16
209 383 5 20
25 383 27 97
201 376 3 16
71 376 5 41
15 376 7 20
235 370 7 34
443 353 7 22
189 352 7 39
143 347 13 57
95 347 3 16
265 346 5 24
159 344 19 61
131 344 7 20
119 344 7 20
105 344 5 20
85 342 5 38
229 340 13 37
201 340 17 25
103 318 5 20
193 259 5 20
297 251 5 20
173 249 5 20
273 232 367 248
215 158 9 22
369 156 124 75
344 128 10 29
607 126 27 32
0 103 2 18
611 72 13 39
479 67 3 16
227 62 7 30
453 54 5 20
585 51 7 21
559 47 5 24
369 45 3 16
131 43 3 17
579 0 5 10
557 0 3 8
461 0 5 11
415 0 3 8
399 0 3 8
341 0 3 8
335 0 3 8
325 0 3 8
307 0 3 8
269 0 3 8
251 0 3 8
223 0 3 8
183 0 3 8
135 0 3 8
125 0 3 8
111 0 3 8
69 0 3 8
63 0 3 8
213 471 3 9
125 471 3 9
81 469 5 11
51 468 5 12
159 463 5 17
175 431 3 17
165 422 3 17
203 421 3 17
59 421 5 20
85 406 5 21
129 399 5 22
29 397 17 83
177 396 7 23
147

215 471 3 9
161 471 3 9
113 471 3 9
107 471 3 9
101 435 5 21
63 434 11 19
219 413 6 36
183 412 11 41
87 402 5 19
0 387 2 18
115 386 11 19
131 377 7 44
13 375 37 105
197 373 15 70
169 371 12 37
0 354 2 18
205 346 3 17
175 345 25 27
213 344 9 21
145 343 19 78
153 385 7 16
139 343 9 21
113 343 15 23
165 340 5 26
91 340 15 39
141 318 5 21
223 298 204 182
454 295 43 75
401 295 50 50
191 290 3 18
0 283 2 19
527 273 113 193
243 251 7 20
191 251 5 20
0 210 2 18
426 192 105 80
369 137 3 16
359 131 3 18
607 126 29 31
609 73 14 37
503 67 3 16
149 60 15 32
587 57 3 16
287 54 3 16
233 54 5 20
575 52 3 17
341 52 3 18
563 46 3 17
341 12 3 16
229 10 5 21
585 0 3 8
545 0 3 8
533 0 3 8
523 0 3 8
485 0 3 8
403 0 3 15
397 0 3 8
347 0 3 8
253 0 3 8
203 0 3 8
101 0 3 8
77 0 3 8
29 0 3 8
21 0 3 8
189 471 3 9
175 471 3 9
129 471 3 9
51 471 3 9
153 459 5 21
79 437 7 23
133 421 5 20
189 412 9 37
166 403 4 18
77 391 3 16
137 385 9 24
209 384 5 24
25 383 21 97
215 382 29 69
173 380 9 41
0 380 2 18
18 377 3 18
199

In [6]:
# cap = cv2.VideoCapture(0)
# if not cap.isOpened():
#     raise IOError("Cannot open webcam")
# while True:
#     ret, rgb_frame = cap.read()
#     rgb_frame = cv2.resize(rgb_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
#     # converting the rgb space to hsv space color
#     hsv_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2HSV)
#     # converting the rgb space to YCbCr space color
#     ycbcr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2YCR_CB)
    
#     # converintg the rgb space to rgba spave 
#     rgba_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2BGRA)
#      # the final mask to extract the binary image
#     mask = multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame)
#     binary_frame = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]) , dtype=np.uint8)
#     binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
#     binary_frame2[mask] = 1
#     binary_frame[mask] = 1
# #   binary_frame = median(binary_frame , disk(2) )
#     h, w = binary_frame.shape[:2]
#     maskk = np.zeros((h+2, w+2), np.uint8)
#     im_floodfill = np.copy(binary_frame)
    
#     # Floodfill from point (0, 0)
#     cv2.floodFill(im_floodfill, maskk, (0,0), 255);

#     # Invert floodfilled image
#     im_floodfill_inv = cv2.bitwise_not(im_floodfill)
 
#     # Combine the two images to get the foreground.
#     im_out = binary_frame | im_floodfill_inv
#     im_out = median(im_out , disk(2) )
#     dilated_img = binary_dilation(im_out , np.ones((8,4))) 
#     im_out =  np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
#     im_out[dilated_img] = 1
#     cv2.imshow('Input', rgb_frame)
#     cv2.imshow('Output2', im_out)
#     cv2.imshow('Output1', binary_frame2)
#     c = cv2.waitKey(1)
#     if c == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()
    